In [25]:
from haystack import Pipeline, component
from haystack.components.builders import PromptBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from haystack.dataclasses import ChatMessage
from typing import List
from haystack.components.builders import ChatPromptBuilder
from haystack.components.joiners import ListJoiner
from mlflow.tracking import MlflowClient
from dotenv import load_dotenv
import os
import json
import mlflow
import requests



2025-12-02 18:57:17 DEBUG [haystack.core.component.component] Registering <class 'haystack.components.joiners.list_joiner.ListJoiner'> as a component
2025-12-02 18:57:17 DEBUG [haystack.core.component.component] Registered Component <class 'haystack.components.joiners.list_joiner.ListJoiner'>


In [3]:
import logging
from haystack import tracing
from haystack.tracing.logging_tracer import LoggingTracer

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.DEBUG)

tracing.tracer.is_content_tracing_enabled = True # to enable tracing/logging content (inputs/outputs)
tracing.enable_tracing(LoggingTracer(tags_color_strings={"haystack.component.input": "\x1b[1;31m", "haystack.component.name": "\x1b[1;34m"}))

In [30]:
chat_message_store = InMemoryChatMessageStore()

In [31]:
load_dotenv()


2025-12-02 19:33:06 WARNI [dotenv.main] python-dotenv could not parse statement starting at line 4
2025-12-02 19:33:06 WARNI [dotenv.main] python-dotenv could not parse statement starting at line 6


True

## Component GroqLLM

In [65]:
@component
class GroqLLM:
    def __init__(self, model_name="meta-llama/llama-4-maverick-17b-128e-instruct", api_key=None):
        self.api_key = os.getenv("GROQ_API_KEY")
        self.model_name = model_name

    @component.output_types(output=List[ChatMessage])
    def run(self, prompt: List[ChatMessage]):
        user_prompt = "".join([msg.text for msg in prompt])
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": user_prompt}],
            "temperature": 0.7,
            "max_tokens": 300
        }

        response = requests.post(url, headers=headers, json=payload)
        try:
            data = response.json()
        except Exception:
            raise ValueError("Gagal parse JSON dari Groq API: ", response.text)

        # Debug untuk melihat isi JSON asli
        if "choices" not in data:
            raise ValueError(
                "Groq API tidak mengembalikan 'choices'.\n"
                f"Status Code: {response.status_code}\n"
                f"Response JSON:\n{json.dumps(data, indent=2)}"
            )

        # Jika OK, ambil isi respon
        result = data["choices"][0]["message"]["content"]
        return {"output": [ChatMessage.from_assistant(result)]}

2025-12-02 19:50:01 DEBUG [haystack.core.component.component] Registering <class '__main__.GroqLLM'> as a component
2025-12-02 19:50:01 DEBUG [haystack.core.component.component] Component __main__.GroqLLM is already registered. Previous imported from '<class '__main__.GroqLLM'>',                 new imported from '<class '__main__.GroqLLM'>'
2025-12-02 19:50:01 DEBUG [haystack.core.component.component] Registered Component <class '__main__.GroqLLM'>


## Predictor

In [66]:
PROMPT_TEMPLATE = """
You are a helpful and friendly assistant. 
Please provide a concise and general answer to the following question:

{% if category == 0 %}
    respond i'm sorry i cannot help you with that
{% else %}
    answer the question based on the input below
{% endif %}

input : {{input}}

output :
"""

In [67]:
@component
class PredictorCategory:
    def __init__(self, model_name , model_tfidf):
        mlflow.set_tracking_uri('sqlite:///mlflow.db')
        self.client = MlflowClient()
        self.model = self._load_model(model_name)
        self.tfidf = self._load_model(model_tfidf)

    @component.output_types(category=str)
    def run(self, input_data: str):
        transform = self.tfidf.transform([input_data])
        category = self.model.predict(transform)
        return {"category": category[0]}
    
    def _load_model(self,model_name:str) :
        version = self.client.search_model_versions(f"name='{model_name}'")
        latest_version = max(version, key=lambda x:int(x.version))
        last_version_number = latest_version.version
        model = mlflow.sklearn.load_model(f"models:/{model_name}/{last_version_number}")
        return model

2025-12-02 19:50:02 DEBUG [haystack.core.component.component] Registering <class '__main__.PredictorCategory'> as a component
2025-12-02 19:50:02 DEBUG [haystack.core.component.component] Component __main__.PredictorCategory is already registered. Previous imported from '<class '__main__.PredictorCategory'>',                 new imported from '<class '__main__.PredictorCategory'>'
2025-12-02 19:50:02 DEBUG [haystack.core.component.component] Registered Component <class '__main__.PredictorCategory'>


In [68]:
@component
class PromptToMessages:
    @component.output_types(messages=list[ChatMessage])
    def run(self, prompt: str):
        # Convert string → List[ChatMessage]
        messages = [
            ChatMessage.from_user(prompt)
        ]
        return {"messages": messages}

2025-12-02 19:50:02 DEBUG [haystack.core.component.component] Registering <class '__main__.PromptToMessages'> as a component
2025-12-02 19:50:02 DEBUG [haystack.core.component.component] Component __main__.PromptToMessages is already registered. Previous imported from '<class '__main__.PromptToMessages'>',                 new imported from '<class '__main__.PromptToMessages'>'
2025-12-02 19:50:02 DEBUG [haystack.core.component.component] Registered Component <class '__main__.PromptToMessages'>


In [69]:
class PipelineCategory:
    def __init__(self):
        self.model = "Logistic_regression_optuna"
        self.tfidf = "tfidf_vectorizer"
        
        self.pipeline = Pipeline()
        self.pipeline.add_component('prediction',PredictorCategory(self.model,self.tfidf))
        self.pipeline.add_component('prompt_builder',PromptBuilder(template=PROMPT_TEMPLATE,required_variables=["input","category"]))
        self.pipeline.add_component('prompt_to_msg',PromptToMessages())
        self.pipeline.add_component('groq_llm',GroqLLM())
        
        self.pipeline.connect("prediction.category","prompt_builder.category")
        self.pipeline.connect("prompt_builder.prompt","prompt_to_msg.prompt")
        self.pipeline.connect("prompt_to_msg.messages","groq_llm")
    
    def run(self, input_data : str) :
        
        res = self.pipeline.run(
            data= {
                "prediction" : {
                    "input_data" : input_data
                },
                "prompt_builder" : {
                    "input" : input_data
                }
                }
        )
        print(f"Pipeline Input : {input_data}")
        return res['groq_llm']['output']
        
            
    
    
        

In [70]:
predictor_pipeline= PipelineCategory()

2025-12-02 19:50:04 DEBUG [haystack.core.pipeline.base] Adding component 'prediction' (<__main__.PredictorCategory object at 0x000002210785AFC0>

Inputs:
  - input_data: str
Outputs:
  - category: str)
2025-12-02 19:50:04 DEBUG [haystack.core.pipeline.base] Adding component 'prompt_builder' (<haystack.components.builders.prompt_builder.PromptBuilder object at 0x0000022102571070>

Inputs:
  - category: Any
  - input: Any
  - template: Optional[str]
  - template_variables: Optional[dict[str, Any]]
Outputs:
  - prompt: str)
2025-12-02 19:50:04 DEBUG [haystack.core.pipeline.base] Adding component 'prompt_to_msg' (<__main__.PromptToMessages object at 0x0000022107859D90>

Inputs:
  - prompt: str
Outputs:
  - messages: list[ChatMessage])
2025-12-02 19:50:04 DEBUG [haystack.core.pipeline.base] Adding component 'groq_llm' (<__main__.GroqLLM object at 0x0000022107F21D90>

Inputs:
  - prompt: List[ChatMessage]
Outputs:
  - output: List[ChatMessage])
2025-12-02 19:50:04 DEBUG [haystack.core.pipeli

## History Tool

In [71]:
class ChatHistoryPipeline:
    def __init__(self, chat_message_store):
        self.chat_message_store = chat_message_store
        self.pipeline = Pipeline()
        self.pipeline.add_component("memory_retriever", ChatMessageRetriever(chat_message_store))
        self.pipeline.add_component("prompt_builder", PromptBuilder(variables=["memories"], required_variables=["memories"], template="""
        Previous Conversations history:
        {% for memory in memories %}
            {{memory.text}}
        {% endfor %}
        """)
        )
        self.pipeline.connect("memory_retriever", "prompt_builder.memories")

    def run(self):
        res = self.pipeline.run(
            data = {},
            include_outputs_from=["prompt_builder"]
        )

        # print("Pipeline Input", res["prompt_builder"]["prompt"])
        return res["prompt_builder"]["prompt"]

        

In [72]:
chat_history_pipeline = ChatHistoryPipeline(chat_message_store=chat_message_store)

2025-12-02 19:50:07 DEBUG [haystack.core.pipeline.base] Adding component 'memory_retriever' (<haystack_experimental.components.retrievers.chat_message_retriever.ChatMessageRetriever object at 0x0000022107876AE0>

Inputs:
  - last_k: Optional[int]
Outputs:
  - messages: List[ChatMessage])
2025-12-02 19:50:07 DEBUG [haystack.core.pipeline.base] Adding component 'prompt_builder' (<haystack.components.builders.prompt_builder.PromptBuilder object at 0x0000022102573290>

Inputs:
  - memories: Any
  - template: Optional[str]
  - template_variables: Optional[dict[str, Any]]
Outputs:
  - prompt: str)
2025-12-02 19:50:07 DEBUG [haystack.core.pipeline.base] Connecting 'memory_retriever.messages' to 'prompt_builder.memories'


## Chatbot AI

In [ ]:
chat_message_writer = ChatMessageWriter(chat_message_store)
while True :
    query = input("Masukkan query: ")
    if query.lower() == "exit":
        break
    
    history = chat_history_pipeline.run()
    messages = [ChatMessage.from_system(history),ChatMessage.from_user(query)]
    chat_message_writer.run([ChatMessage.from_user(query)])
    response = predictor_pipeline.run(query)
    response_text = response[0]._content[0].text
    
    messsages_save= [ChatMessage.from_assistant(response_text)]
    chat_message_writer.run(messages=messsages_save)
        
    print(f"Response: {response_text}")
    
    
    

2025-12-02 19:55:49 INFO  [haystack.core.pipeline.pipeline] Running component memory_retriever
2025-12-02 19:55:49 DEBUG [haystack.tracing.logging_tracer] Operation: haystack.component.run
2025-12-02 19:55:49 DEBUG [haystack.tracing.logging_tracer] haystack.component.name=memory_retriever
2025-12-02 19:55:49 DEBUG [haystack.tracing.logging_tracer] haystack.component.type=ChatMessageRetriever
2025-12-02 19:55:49 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_types={"last_k": "NoneType"}
2025-12-02 19:55:49 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_spec={"last_k": {"type": "typing.Optional[int]", "senders": []}}
2025-12-02 19:55:49 DEBUG [haystack.tracing.logging_tracer] haystack.component.output_spec={"messages": {"type": "typing.List[haystack.dataclasses.chat_message.ChatMessage]", "receivers": ["prompt_builder"]}}
2025-12-02 19:55:49 DEBUG [haystack.tracing.logging_tracer] haystack.component.input={"last_k": null}
2025-12-02 19:55:49 DEBUG [hay

Pipeline Input : halo
Response: Halo! It seems like you're saying hello. How can I assist you today?


2025-12-02 19:56:16 INFO  [haystack.core.pipeline.pipeline] Running component memory_retriever
2025-12-02 19:56:16 DEBUG [haystack.tracing.logging_tracer] Operation: haystack.component.run
2025-12-02 19:56:16 DEBUG [haystack.tracing.logging_tracer] haystack.component.name=memory_retriever
2025-12-02 19:56:16 DEBUG [haystack.tracing.logging_tracer] haystack.component.type=ChatMessageRetriever
2025-12-02 19:56:16 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_types={"last_k": "NoneType"}
2025-12-02 19:56:16 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_spec={"last_k": {"type": "typing.Optional[int]", "senders": []}}
2025-12-02 19:56:16 DEBUG [haystack.tracing.logging_tracer] haystack.component.output_spec={"messages": {"type": "typing.List[haystack.dataclasses.chat_message.ChatMessage]", "receivers": ["prompt_builder"]}}
2025-12-02 19:56:16 DEBUG [haystack.tracing.logging_tracer] haystack.component.input={"last_k": null}
2025-12-02 19:56:16 DEBUG [hay

Pipeline Input : can you describe how the perform goverment modi
Response: The Indian government, led by Prime Minister Narendra Modi, has implemented various policies and programs aimed at promoting economic growth, improving infrastructure, and enhancing the overall quality of life for its citizens. Some key initiatives include:

1. **Economic Reforms**: Implementing policies to boost economic growth, such as Goods and Services Tax (GST) and Insolvency and Bankruptcy Code.
2. **Infrastructure Development**: Investing in infrastructure projects like roads, highways, and railways to improve connectivity.
3. **Social Welfare Programs**: Launching initiatives like Jan Dhan Yojana, Ujjwala Yojana, and Swachh Bharat Abhiyan to promote financial inclusion, women's empowerment, and sanitation.
4. **Digital India**: Promoting digital literacy and online services to enhance governance and connectivity.

These are just a few examples of the government's efforts under PM Modi's leadership.


2025-12-02 19:56:36 INFO  [haystack.core.pipeline.pipeline] Running component memory_retriever
2025-12-02 19:56:36 DEBUG [haystack.tracing.logging_tracer] Operation: haystack.component.run
2025-12-02 19:56:36 DEBUG [haystack.tracing.logging_tracer] haystack.component.name=memory_retriever
2025-12-02 19:56:36 DEBUG [haystack.tracing.logging_tracer] haystack.component.type=ChatMessageRetriever
2025-12-02 19:56:36 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_types={"last_k": "NoneType"}
2025-12-02 19:56:36 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_spec={"last_k": {"type": "typing.Optional[int]", "senders": []}}
2025-12-02 19:56:36 DEBUG [haystack.tracing.logging_tracer] haystack.component.output_spec={"messages": {"type": "typing.List[haystack.dataclasses.chat_message.ChatMessage]", "receivers": ["prompt_builder"]}}
2025-12-02 19:56:36 DEBUG [haystack.tracing.logging_tracer] haystack.component.input={"last_k": null}
2025-12-02 19:56:36 DEBUG [hay

Pipeline Input : i hate modi
Response: I'm sorry I cannot help you with that.


2025-12-02 19:56:43 INFO  [haystack.core.pipeline.pipeline] Running component memory_retriever
2025-12-02 19:56:43 DEBUG [haystack.tracing.logging_tracer] Operation: haystack.component.run
2025-12-02 19:56:43 DEBUG [haystack.tracing.logging_tracer] haystack.component.name=memory_retriever
2025-12-02 19:56:43 DEBUG [haystack.tracing.logging_tracer] haystack.component.type=ChatMessageRetriever
2025-12-02 19:56:43 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_types={"last_k": "NoneType"}
2025-12-02 19:56:43 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_spec={"last_k": {"type": "typing.Optional[int]", "senders": []}}
2025-12-02 19:56:43 DEBUG [haystack.tracing.logging_tracer] haystack.component.output_spec={"messages": {"type": "typing.List[haystack.dataclasses.chat_message.ChatMessage]", "receivers": ["prompt_builder"]}}
2025-12-02 19:56:43 DEBUG [haystack.tracing.logging_tracer] haystack.component.input={"last_k": null}
2025-12-02 19:56:43 DEBUG [hay

Pipeline Input : fuck you
Response: I'm sorry, I cannot help you with that.


2025-12-02 19:56:54 INFO  [haystack.core.pipeline.pipeline] Running component memory_retriever
2025-12-02 19:56:54 DEBUG [haystack.tracing.logging_tracer] Operation: haystack.component.run
2025-12-02 19:56:54 DEBUG [haystack.tracing.logging_tracer] haystack.component.name=memory_retriever
2025-12-02 19:56:54 DEBUG [haystack.tracing.logging_tracer] haystack.component.type=ChatMessageRetriever
2025-12-02 19:56:54 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_types={"last_k": "NoneType"}
2025-12-02 19:56:54 DEBUG [haystack.tracing.logging_tracer] haystack.component.input_spec={"last_k": {"type": "typing.Optional[int]", "senders": []}}
2025-12-02 19:56:54 DEBUG [haystack.tracing.logging_tracer] haystack.component.output_spec={"messages": {"type": "typing.List[haystack.dataclasses.chat_message.ChatMessage]", "receivers": ["prompt_builder"]}}
2025-12-02 19:56:54 DEBUG [haystack.tracing.logging_tracer] haystack.component.input={"last_k": null}
2025-12-02 19:56:54 DEBUG [hay

Pipeline Input : hate modi
Response: I'm sorry, I cannot help you with that.


In [78]:
print(history)


        Previous Conversations history:
        
            halo
        
            Halo! It seems like you're saying hello. How can I assist you today?
        
            can you describe how the perform goverment modi
        
            The Indian government, led by Prime Minister Narendra Modi, has implemented various policies and programs aimed at promoting economic growth, improving infrastructure, and enhancing the overall quality of life for its citizens. Some key initiatives include:

1. **Economic Reforms**: Implementing policies to boost economic growth, such as Goods and Services Tax (GST) and Insolvency and Bankruptcy Code.
2. **Infrastructure Development**: Investing in infrastructure projects like roads, highways, and railways to improve connectivity.
3. **Social Welfare Programs**: Launching initiatives like Jan Dhan Yojana, Ujjwala Yojana, and Swachh Bharat Abhiyan to promote financial inclusion, women's empowerment, and sanitation.
4. **Digital India**: Promoti